In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplcursors
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [6]:
scaler = MinMaxScaler()

In [ ]:
#excluindo nulos e renomeando algumas colunas do dataset    
caminho_arquivo = './data/data.xlsx'
df = pd.read_excel(caminho_arquivo)
df = df.rename(columns={'Unnamed: 1': 'DRS', 'Unnamed: 3': 'GVE', 'Unnamed: 4': 'Codigo_regiao', 'Unnamed: 5': 'Regiao', 'Unnamed: 6': 'Municipio'})
df = df.dropna()
df['Municipio'] = df['Municipio'].replace(to_replace=r'\d', value='', regex=True)
semana = 1

In [9]:
#renomeando as colunas do dataset
for i in range(7, 60):
    if i == 59:
        df = df.rename(columns={f'Unnamed: {i}': 'Total'})
    else:
        df = df.rename(columns={f'Unnamed: {i}': semana})
        semana += 1

In [10]:
linha_sp = df.loc[df['Regiao'] == 'SÃO PAULO']
semanas_infectados = pd.DataFrame()